## Import Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import requests
import io
import csv
import numpy as np

## Load Dataset

In [ ]:
# Pass in the path to the training set csv file you downloaded

df = pd.read_csv('training_set_v1.csv')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df['CUSTOMER_ID'].nunique()

In [ ]:
df['SUBSCRIBED'].value_counts()

## Impute New Features

In [ ]:
datetime.date.today().year

In [ ]:
df['AGE'] = datetime.date.today().year - df['BIRTH_YEAR']

In [ ]:
df.head()

In [ ]:
df['NUM_WEB_PURCHASES'].hist()

In [ ]:
df['NUM_WEB_PURCHASES'].describe()

In [ ]:
def web_purchase_bin(x):
    if x < 88:
        return 1
    elif 88 <= x <= 107:
        return 2
    elif 108 <= x <= 128:
        return 3
    elif x >= 129:
        return 4

In [ ]:
df['NUM_WEB_PURCHASES_BIN'] = df['NUM_WEB_PURCHASES'].apply(lambda x: web_purchase_bin(x))

In [ ]:
df['NUM_WEB_PURCHASES_BIN'].isnull().sum()

In [ ]:
df.head()

In [ ]:
df['NUM_WEB_PURCHASES_BIN'].hist()

## APIs

In [ ]:
df.head()

In [ ]:
! pip install pygeocodio

In [ ]:
from geocodio import GeocodioClient

geocodio_api = 'YOUR API KEY'
client = GeocodioClient(geocodio_api)

In [ ]:
df['FULL_ADDRESS'] = df['ADDRESS1'] + ' ' + df['CITY'] + ' ' + df['STATE'] + ' ' + df['ZIPCODE'].astype(str)
df.dropna(subset=['FULL_ADDRESS'], inplace=True)

In [ ]:
# format for API

api_dict = {}

for index, row in df.iterrows():
    api_dict[str(row['CUSTOMER_ID'])] = row['FULL_ADDRESS']

In [ ]:
len(api_dict)

In [ ]:
api_dict

In [ ]:
geocoded_addresses = client.geocode(api_dict, None, fields=['census2022','acs-demographics', 'acs-social', 'acs-families'])

In [ ]:
geocoded_addresses['63631443']['results'][0]

In [ ]:
def percent_family(customer_id):
    # census data doesn't have 100% coverage, return null if missing this field
    try:
        return geocoded_addresses[customer_id]['results'][0]['fields']['acs']['families']['Household type by household']['Family households']['percentage']
    except:
        return np.nan

In [ ]:
def percent_female_bachelor(customer_id):
    # census data doesn't have 100% coverage, return null if missing this field
    try:
        return geocoded_addresses[customer_id]['results'][0]['fields']['acs']['social']['Population by minimum level of education']["Female: Bachelor's degree"]['percentage']
    except:
        return np.nan

In [ ]:
df['PERCENT_FAMILY'] = df['CUSTOMER_ID'].apply(lambda x: percent_family(str(x)))
df['PERCENT_FEMALE_BACHELOR'] = df['CUSTOMER_ID'].apply(lambda x: percent_female_bachelor(str(x)))

In [ ]:
# check coverage
print((df['PERCENT_FAMILY'].isnull().sum()/df.shape[0]) * 100)
print((df['PERCENT_FEMALE_BACHELOR'].isnull().sum()/df.shape[0]) * 100)

In [ ]:
df.head()

### Public Alternative

In [ ]:
! pip install urllib3==1.26.15 requests-toolbelt==0.10.1
! pip install censusgeocode

In [ ]:
df.head()

In [ ]:
census_df = df[['ADDRESS1', 'CITY', 'STATE', 'ZIPCODE']]

In [ ]:
census_df.head()

In [ ]:
# save as CSV file
census_df.to_csv('census_df.csv', header=False)

In [ ]:
import censusgeocode

cg = censusgeocode.CensusGeocode(benchmark='Public_AR_Current',vintage='Current_Current')
k = cg.addressbatch('census_df.csv')

# Bonus
# Get clean output in Dataframe

df_output = pd.DataFrame(k, columns=k[0].keys())

In [ ]:
df_output.head()

In [ ]:
df_output['match'].value_counts()

In [ ]:
! pip install census

In [ ]:
from census import Census

api_key = 'YOUR US CENSUS API KEY'
c = Census(api_key)
# https://api.census.gov/data/2022/acs/acs1/variables.html
# percent family household = 'B11016_002E'/ 'B11016_001E'

variable_to_extract = "B11016_002E,B11016_001E"

resp = c.acs5.get(fields=('NAME', variable_to_extract),
           geo={'for': 'zip code tabulation area:*'}, 
           year=2021)

In [ ]:
resp

In [ ]:
# create zipcode: percent_family lookup dictionary

cleaned_dict = {}

for dictionary in resp:
    zipcode = dictionary['zip code tabulation area']
    total_households = dictionary["B11016_001E"]
    family_households = dictionary["B11016_002E"]
    if total_households != 0:
        percent_family = family_households/total_households
        cleaned_dict[zipcode] = percent_family 
    else:
        cleaned_dict[zipcode] = np.nan 
    

In [ ]:
cleaned_dict

In [ ]:
df['ZIPCODE'] = df['ZIPCODE'].astype(str)
df['ZIPCODE'] = df['ZIPCODE'].apply(lambda x: '0'+x if len(x)==4 else x)
df['PERCENT_FAMILY_USCENSUS'] = df['ZIPCODE'].apply(lambda x: cleaned_dict[x])

In [ ]:
df.head()

## Correlation?

In [ ]:
def gen_corr_grid(df, depvar):
    corr_dict = {
            'variable': [],
            'corr': []
        }
    for col in list(df.columns):
        if df[col].dtype not in ['object', '<M8[ns]'] and col != depvar :
            r = df[depvar].corr(df[col])   
            corr_dict['variable'].append(col)
            corr_dict['corr'].append(r)
    return pd.DataFrame(corr_dict).sort_values(by=['corr'], ascending=False)


df_corrs = gen_corr_grid(df[['CD_GENDER', 'CD_CREDIT_RATING', 'HD_INCOME_BAND_SK',\
                             'AVG_DOLLAR_ITEM', 'WEB_RETURN_RATE', 'AGE', 'NUM_WEB_PURCHASES_BIN',\
                             'PERCENT_FAMILY', 'PERCENT_FEMALE_BACHELOR', 'PERCENT_FAMILY_USCENSUS', 'SUBSCRIBED']],
                         'SUBSCRIBED')
df_corrs.head(20)